In [7]:
from pycryptosat import Solver
import os

m = Solver()

# 一共有9(9行)*4(每个数在1-9之间，需要4位表示)*9(9列)=324个变量
# 添加所有变量在1-9之间的约束
for i in range(9):
    for j in range(9):
        # 数独中第j(坐标y)行第i(坐标x)列的元素
        m.add_clause([-(i * 4 + j * 9 * 4 + 1), -(i * 4 + j * 9 * 4 + 2)])
        m.add_clause([-(i * 4 + j * 9 * 4 + 1), -(i * 4 + j * 9 * 4 + 3)])
        m.add_clause([i * 4 + j * 9 * 4 + 1, i * 4 + j * 9 * 4 + 2, i * 4 + j * 9 * 4 + 3, i * 4 + j * 9 * 4 + 4])

# 添加所有行内数字不相同
# 这里需要辅助变量
assVar = 324
for row in range(9):
    for i in range(9):
        for j in range(i + 1, 9):
            # 第row行的第i个变量和第j个变量不等
            m.add_xor_clause([row * 9 * 4 + i * 4 + 1, row * 9 * 4 + j * 4 + 1, assVar + 1], False)
            m.add_xor_clause([row * 9 * 4 + i * 4 + 2, row * 9 * 4 + j * 4 + 2, assVar + 2], False)
            m.add_xor_clause([row * 9 * 4 + i * 4 + 3, row * 9 * 4 + j * 4 + 3, assVar + 3], False)
            m.add_xor_clause([row * 9 * 4 + i * 4 + 4, row * 9 * 4 + j * 4 + 4, assVar + 4], False)
            m.add_clause([assVar + 1, assVar + 2, assVar + 3, assVar + 4])
            assVar = assVar + 4
# 添加所有列内数字不同
for col in range(9):
    for i in range(9):
        for j in range(i + 1, 9):
            # 第col列的第i个变量和第j个变量不等
            m.add_xor_clause([i * 9 * 4 + col * 4 + 1, j * 9 * 4 + col * 4 + 1, assVar + 1], False)
            m.add_xor_clause([i * 9 * 4 + col * 4 + 2, j * 9 * 4 + col * 4 + 2, assVar + 2], False)
            m.add_xor_clause([i * 9 * 4 + col * 4 + 3, j * 9 * 4 + col * 4 + 3, assVar + 3], False)
            m.add_xor_clause([i * 9 * 4 + col * 4 + 4, j * 9 * 4 + col * 4 + 4, assVar + 4], False)
            m.add_clause([assVar + 1, assVar + 2, assVar + 3, assVar + 4])
            assVar = assVar + 4
# 添加所有3*3方框内数字不同
for x in range(3):
    for y in range(3):
        for i in range(9):
            for j in range(i + 1, 9):
                # 第[x, y]个方框内的第i个变量和第j个变量不等
                i_x = i % 3
                i_y = i // 3
                j_x = j % 3
                j_y = j // 3
                m.add_xor_clause([y * 3 * 9 * 4 + x * 3 * 4 + i_y * 9 * 4 + i_x * 4 + 1, y * 3 * 9 * 4 + x * 3 * 4 + j_y * 9 * 4 + j_x * 4 + 1, assVar + 1], False)
                m.add_xor_clause([y * 3 * 9 * 4 + x * 3 * 4 + i_y * 9 * 4 + i_x * 4 + 2, y * 3 * 9 * 4 + x * 3 * 4 + j_y * 9 * 4 + j_x * 4 + 2, assVar + 2], False)
                m.add_xor_clause([y * 3 * 9 * 4 + x * 3 * 4 + i_y * 9 * 4 + i_x * 4 + 3, y * 3 * 9 * 4 + x * 3 * 4 + j_y * 9 * 4 + j_x * 4 + 3, assVar + 3], False)
                m.add_xor_clause([y * 3 * 9 * 4 + x * 3 * 4 + i_y * 9 * 4 + i_x * 4 + 4, y * 3 * 9 * 4 + x * 3 * 4 + j_y * 9 * 4 + j_x * 4 + 4, assVar + 4], False)
                m.add_clause([assVar + 1, assVar + 2, assVar + 3, assVar + 4])
                assVar = assVar + 4

# 输入数独
problem = []
for i in range(1, 10):
    inputRow = input("第" + str(i) + "行(空格划分数字，无数字用0代替)：").split(" ")
    if len(inputRow) != 9:
        print("长度错误")
        os._exit(0)
    problem.append([int(inputRow[i]) for i in range(len(inputRow))])
# 添加数独约束
# x = [x0, x1, x2, x3] = x0 * 8 + x1 * 4 + x2 * 2 + x3 * 1
for row in range(9):
    for col in range(9):
        if(problem[row][col]):
            for i in range(4):
                x = problem[row][col] // (2 ** (3 - i))
                problem[row][col] = problem[row][col] % (2 ** (3 - i))
                if(x):
                    m.add_clause([row * 9 * 4 + col * 4 + i + 1])
                else:
                    m.add_clause([-(row * 9 * 4 + col * 4 + i + 1)])

sat, solution = m.solve()

if(sat == False):
    print("无解")
else:
    for row in range(9):
        for col in range(9):
            print(solution[row * 4 * 9 + col * 4 + 1] * 8 + solution[row * 4 * 9 + col * 4 + 2] * 4 + solution[row * 4 * 9 + col * 4 + 3] * 2 + solution[row * 4 * 9 + col * 4 + 4] * 1, end="  ")
        print()

第1行(空格划分数字，无数字用0代替)：8 0 4 0 0 0 5 0 6
第2行(空格划分数字，无数字用0代替)：0 0 0 1 0 0 0 0 0
第3行(空格划分数字，无数字用0代替)：3 0 1 0 0 2 0 0 7
第4行(空格划分数字，无数字用0代替)：0 1 0 0 4 0 7 0 0
第5行(空格划分数字，无数字用0代替)：0 0 0 9 0 7 0 0 0
第6行(空格划分数字，无数字用0代替)：0 0 5 0 8 0 0 3 0
第7行(空格划分数字，无数字用0代替)：1 0 0 6 0 0 2 0 8
第8行(空格划分数字，无数字用0代替)：0 0 0 0 0 1 0 0 0
第9行(空格划分数字，无数字用0代替)：4 0 7 0 0 0 1 0 5
8  2  4  7  3  9  5  1  6  
5  7  9  1  6  8  4  2  3  
3  6  1  4  5  2  8  9  7  
9  1  6  3  4  5  7  8  2  
2  3  8  9  1  7  6  5  4  
7  4  5  2  8  6  9  3  1  
1  5  3  6  9  4  2  7  8  
6  8  2  5  7  1  3  4  9  
4  9  7  8  2  3  1  6  5  
